# Chapter 41: Thread-Safe Patterns

This notebook covers thread-safe data structures and patterns for concurrent programming. You will learn how to use `Queue` for safe inter-thread communication, implement the producer-consumer pattern, manage per-thread state with `threading.local`, and control queue capacity.

## Key Concepts
- **`Queue`**: Thread-safe FIFO queue for passing data between threads
- **Producer-consumer pattern**: Decouple data production from consumption using a queue
- **Sentinel values**: Signal termination to consumers via a special value (e.g., `None`)
- **`threading.local`**: Per-thread storage where each thread sees its own data
- **Queue `maxsize`**: Limit queue capacity to apply backpressure on producers

## Section 1: Queue Basics -- Thread-Safe FIFO

The `queue.Queue` class provides a thread-safe FIFO (first-in, first-out) data structure. Multiple threads can safely `put()` and `get()` items without additional locking.

In [ ]:
from queue import Queue

# Basic FIFO queue
q: Queue[int] = Queue()

# Put items into the queue
q.put(1)
q.put(2)
q.put(3)

# Get items in FIFO order
first: int = q.get()
second: int = q.get()
third: int = q.get()

print(f"First: {first}")
print(f"Second: {second}")
print(f"Third: {third}")
print(f"FIFO order correct: {first == 1 and second == 2 and third == 3}")

In [ ]:
from queue import Queue

# Queue utility methods
q: Queue[str] = Queue()

print(f"Empty at start: {q.empty()}")
print(f"Size at start: {q.qsize()}")

q.put("alpha")
q.put("beta")

print(f"\nAfter adding 2 items:")
print(f"Empty: {q.empty()}")
print(f"Size: {q.qsize()}")

# task_done() and join() for tracking completion
item: str = q.get()
q.task_done()  # Signal that the item has been processed
print(f"\nProcessed: {item}")
print(f"Remaining size: {q.qsize()}")

## Section 2: Queue Timeout and Empty Exception

By default, `Queue.get()` blocks until an item is available. You can pass a `timeout` parameter to raise `queue.Empty` if no item arrives within the timeout period.

In [ ]:
from queue import Empty, Queue

q: Queue[int] = Queue()

# get() with timeout raises Empty if the queue is empty
try:
    q.get(timeout=0.01)
    print("Got an item (unexpected)")
except Empty:
    print("Caught Empty exception -- queue was empty after timeout")

# Non-blocking get with block=False
try:
    q.get(block=False)
    print("Got an item (unexpected)")
except Empty:
    print("Caught Empty exception -- non-blocking get on empty queue")

In [ ]:
from queue import Empty, Queue
import threading
import time

# Practical pattern: drain a queue with timeout
q: Queue[int] = Queue()
collected: list[int] = []


def delayed_producer() -> None:
    """Add items to the queue with small delays."""
    for i in range(3):
        time.sleep(0.02)
        q.put(i)


t: threading.Thread = threading.Thread(target=delayed_producer)
t.start()

# Collect items with timeout-based polling
while True:
    try:
        item: int = q.get(timeout=0.1)
        collected.append(item)
    except Empty:
        # No more items within timeout
        break

t.join()

print(f"Collected items: {collected}")
print(f"All items received: {collected == [0, 1, 2]}")

## Section 3: Producer-Consumer Pattern

The producer-consumer pattern decouples threads that generate data (producers) from threads that process data (consumers). A `Queue` serves as the buffer between them. A sentinel value (typically `None`) signals the consumer to stop.

In [ ]:
import threading
from queue import Queue

q: Queue[int | None] = Queue()
results: list[int] = []
lock: threading.Lock = threading.Lock()


def producer() -> None:
    """Produce 5 items, then send a sentinel to stop the consumer."""
    for i in range(5):
        q.put(i)
    q.put(None)  # Sentinel signals end of data


def consumer() -> None:
    """Consume items until the sentinel is received."""
    while True:
        item: int | None = q.get()
        if item is None:
            break
        with lock:
            results.append(item)


t1: threading.Thread = threading.Thread(target=producer)
t2: threading.Thread = threading.Thread(target=consumer)
t1.start()
t2.start()
t1.join()
t2.join()

print(f"Consumed results: {sorted(results)}")
print(f"Expected: [0, 1, 2, 3, 4]")
print(f"Correct: {sorted(results) == [0, 1, 2, 3, 4]}")

In [ ]:
import threading
from queue import Queue

# Multiple producers and consumers
q: Queue[int | None] = Queue()
results: list[int] = []
lock: threading.Lock = threading.Lock()
NUM_PRODUCERS: int = 2
NUM_CONSUMERS: int = 2
ITEMS_PER_PRODUCER: int = 5


def multi_producer(producer_id: int) -> None:
    """Produce items tagged with producer ID."""
    for i in range(ITEMS_PER_PRODUCER):
        q.put(producer_id * 100 + i)


def multi_consumer() -> None:
    """Consume items until sentinel."""
    while True:
        item: int | None = q.get()
        if item is None:
            break
        with lock:
            results.append(item)


# Start producers
producers: list[threading.Thread] = [
    threading.Thread(target=multi_producer, args=(pid,))
    for pid in range(NUM_PRODUCERS)
]
for p in producers:
    p.start()
for p in producers:
    p.join()

# Send one sentinel per consumer
for _ in range(NUM_CONSUMERS):
    q.put(None)

# Start consumers
consumers: list[threading.Thread] = [
    threading.Thread(target=multi_consumer) for _ in range(NUM_CONSUMERS)
]
for c in consumers:
    c.start()
for c in consumers:
    c.join()

print(f"Total items consumed: {len(results)}")
print(f"Expected: {NUM_PRODUCERS * ITEMS_PER_PRODUCER}")
print(f"All items accounted for: {len(results) == NUM_PRODUCERS * ITEMS_PER_PRODUCER}")

## Section 4: Queue maxsize -- Backpressure

A `Queue` can be created with a `maxsize` to limit the number of items it can hold. When the queue is full, `put()` blocks until space is available. This provides natural backpressure on fast producers.

In [ ]:
from queue import Queue

# Create a queue with maximum size of 2
q: Queue[int] = Queue(maxsize=2)

q.put(1)
q.put(2)

print(f"Queue size: {q.qsize()}")
print(f"Queue full: {q.full()}")
print(f"Max size: {q.maxsize}")

# Remove one item to make room
removed: int = q.get()
print(f"\nRemoved: {removed}")
print(f"Queue full after get: {q.full()}")
print(f"Queue size: {q.qsize()}")

In [ ]:
from queue import Full, Queue

# Non-blocking put on a full queue
q: Queue[int] = Queue(maxsize=2)
q.put(1)
q.put(2)

# Try to put without blocking
try:
    q.put(3, block=False)
    print("Put succeeded (unexpected)")
except Full:
    print("Caught Full exception -- queue at maxsize")

# Try with timeout
try:
    q.put(3, timeout=0.01)
    print("Put succeeded (unexpected)")
except Full:
    print("Caught Full exception -- queue still full after timeout")

print(f"\nQueue size: {q.qsize()}")
print(f"Queue full: {q.full()}")

In [ ]:
import threading
import time
from queue import Queue

# Backpressure in action: fast producer, slow consumer
q: Queue[int | None] = Queue(maxsize=3)
produced_at: list[float] = []
consumed_at: list[float] = []
start_time: float = time.perf_counter()


def fast_producer() -> None:
    """Produce items as fast as possible."""
    for i in range(6):
        q.put(i)  # Blocks when queue is full
        produced_at.append(time.perf_counter() - start_time)
    q.put(None)


def slow_consumer() -> None:
    """Consume items slowly."""
    while True:
        item: int | None = q.get()
        if item is None:
            break
        consumed_at.append(time.perf_counter() - start_time)
        time.sleep(0.03)  # Slow processing


t1: threading.Thread = threading.Thread(target=fast_producer)
t2: threading.Thread = threading.Thread(target=slow_consumer)
t1.start()
t2.start()
t1.join()
t2.join()

print("Backpressure demonstration:")
print(f"Items produced: {len(produced_at)}")
print(f"Items consumed: {len(consumed_at)}")
print(f"Producer was slowed by full queue (backpressure applied)")

## Section 5: Thread-Local Storage

`threading.local()` creates an object where each thread sees its own independent copy of the data. This is useful for per-thread caches, database connections, or other state that should not be shared.

In [ ]:
import threading
import time

# Each thread gets its own copy of local.data
local: threading.local = threading.local()
results: list[int] = []
lock: threading.Lock = threading.Lock()


def worker(value: int) -> None:
    """Set thread-local data and verify it persists."""
    local.data = value
    time.sleep(0.01)  # Let other threads run
    # Each thread still sees its own value
    with lock:
        results.append(local.data)


threads: list[threading.Thread] = [
    threading.Thread(target=worker, args=(i,)) for i in range(4)
]
for t in threads:
    t.start()
for t in threads:
    t.join()

print(f"Results (sorted): {sorted(results)}")
print(f"Each thread kept its own value: {sorted(results) == [0, 1, 2, 3]}")

In [ ]:
import threading

# Thread-local storage with multiple attributes
local: threading.local = threading.local()
reports: list[str] = []
lock: threading.Lock = threading.Lock()


def database_worker(worker_id: int, db_name: str) -> None:
    """Simulate a worker with its own database connection."""
    local.worker_id = worker_id
    local.db_name = db_name
    local.query_count = 0

    # Simulate queries
    for _ in range(3):
        local.query_count += 1

    report: str = (
        f"Worker {local.worker_id} on {local.db_name}: "
        f"{local.query_count} queries"
    )
    with lock:
        reports.append(report)


threads: list[threading.Thread] = [
    threading.Thread(target=database_worker, args=(i, f"db_{i}"))
    for i in range(3)
]
for t in threads:
    t.start()
for t in threads:
    t.join()

for report in sorted(reports):
    print(report)

In [ ]:
import threading

# Thread-local attributes are not shared
local: threading.local = threading.local()
local.main_value = "set in main thread"

child_saw: list[str] = []


def child_thread() -> None:
    """Check if the child thread can see the main thread's local data."""
    has_attr: bool = hasattr(local, "main_value")
    child_saw.append(f"has main_value: {has_attr}")

    # Set our own value
    local.child_value = "set in child thread"
    child_saw.append(f"child_value: {local.child_value}")


t: threading.Thread = threading.Thread(target=child_thread)
t.start()
t.join()

for msg in child_saw:
    print(f"Child thread: {msg}")

print(f"\nMain thread: main_value = {local.main_value}")
print(f"Main thread: has child_value = {hasattr(local, 'child_value')}")

## Section 6: Practical Patterns -- Pipeline with Queues

Queues can be chained together to form processing pipelines where each stage runs in its own thread.

In [ ]:
import threading
from queue import Queue

# Two-stage pipeline: generate -> transform -> collect
stage1_q: Queue[int | None] = Queue()
stage2_q: Queue[str | None] = Queue()
final_results: list[str] = []


def generator() -> None:
    """Stage 1: Generate raw numbers."""
    for i in range(5):
        stage1_q.put(i)
    stage1_q.put(None)


def transformer() -> None:
    """Stage 2: Transform numbers to formatted strings."""
    while True:
        item: int | None = stage1_q.get()
        if item is None:
            stage2_q.put(None)
            break
        stage2_q.put(f"item-{item:03d}")


def collector() -> None:
    """Stage 3: Collect final results."""
    while True:
        item: str | None = stage2_q.get()
        if item is None:
            break
        final_results.append(item)


threads: list[threading.Thread] = [
    threading.Thread(target=generator),
    threading.Thread(target=transformer),
    threading.Thread(target=collector),
]
for t in threads:
    t.start()
for t in threads:
    t.join()

print(f"Pipeline results: {final_results}")
print(f"Expected 5 items: {len(final_results) == 5}")

In [ ]:
import threading
from queue import Queue

# Worker pool pattern: multiple workers processing from a shared queue
task_queue: Queue[int | None] = Queue()
results: list[int] = []
lock: threading.Lock = threading.Lock()
NUM_WORKERS: int = 3


def pool_worker(worker_id: int) -> None:
    """Process tasks from the shared queue."""
    while True:
        task: int | None = task_queue.get()
        if task is None:
            break
        result: int = task * task  # Square the number
        with lock:
            results.append(result)


# Add tasks
for i in range(10):
    task_queue.put(i)

# Add sentinel for each worker
for _ in range(NUM_WORKERS):
    task_queue.put(None)

# Start workers
workers: list[threading.Thread] = [
    threading.Thread(target=pool_worker, args=(wid,))
    for wid in range(NUM_WORKERS)
]
for w in workers:
    w.start()
for w in workers:
    w.join()

print(f"Results (sorted): {sorted(results)}")
expected: list[int] = [i * i for i in range(10)]
print(f"Expected: {expected}")
print(f"Correct: {sorted(results) == expected}")

## Section 7: Other Queue Types

The `queue` module also provides `LifoQueue` (stack) and `PriorityQueue` for different ordering needs.

In [ ]:
from queue import LifoQueue, PriorityQueue

# LifoQueue -- last-in, first-out (stack behavior)
lifo: LifoQueue[str] = LifoQueue()
lifo.put("first")
lifo.put("second")
lifo.put("third")

print("LifoQueue (stack order):")
print(f"  {lifo.get()}")  # third
print(f"  {lifo.get()}")  # second
print(f"  {lifo.get()}")  # first

# PriorityQueue -- lowest value first
pq: PriorityQueue[tuple[int, str]] = PriorityQueue()
pq.put((3, "low priority"))
pq.put((1, "high priority"))
pq.put((2, "medium priority"))

print("\nPriorityQueue (lowest value first):")
while not pq.empty():
    priority, message = pq.get()
    print(f"  Priority {priority}: {message}")

## Summary

### Queue Basics
- **`Queue()`**: Thread-safe FIFO queue; `put()` adds items, `get()` removes them in order
- **`Queue(maxsize=N)`**: Limits capacity; `put()` blocks when full, providing backpressure
- **`q.full()`** / **`q.empty()`** / **`q.qsize()`**: Check queue state (approximate in threaded code)
- **`q.get(timeout=N)`**: Raises `queue.Empty` if no item is available within `N` seconds

### Producer-Consumer Pattern
- Producers call `q.put(item)` to add work; consumers call `q.get()` to retrieve it
- Use a **sentinel value** (e.g., `None`) to signal consumers to stop
- For multiple consumers, send one sentinel per consumer

### Thread-Local Storage
- **`threading.local()`**: Each thread gets its own independent copy of attributes
- Attributes set in one thread are invisible to other threads
- Useful for per-thread caches, connections, or context

### Additional Queue Types
- **`LifoQueue`**: Last-in, first-out (stack) ordering
- **`PriorityQueue`**: Items are retrieved in priority order (lowest value first)

### Best Practices
- Always use sentinel values or `task_done()` / `join()` for clean shutdown
- Use `maxsize` to prevent unbounded memory growth from fast producers
- Prefer `threading.local()` over manual per-thread dictionaries for thread-specific state